In [2]:
from dbconn import querydb, querydbtopandas 
import numpy as np
import pandas as pd
import datetime
import os

import plotly.graph_objects as go

In [ ]:
prestador = 660

In [22]:
temporal = pd.DataFrame(columns= ["farmacia"], data= ["desvio"])
fechas = [i for i in os.listdir('datos_farmacia/') if len(i) == 12]
fechas.sort()

for i in fechas:
    df = pd.read_parquet(f'datos_farmacia/{i}', columns =['N° Prestador', "Médico", "Especialidad", "Media por especialidad", "Afil. Usuarios","Recetas", "Promedio por afiliado"  ] )
    if len(df[df["N° Prestador"] == prestador]) == 0:
        temporal[i[:7]] = 0 
    else:
        especialidad = df[df["N° Prestador"] == prestador].iloc[0,2]

        desvio = df[df["Especialidad"] == especialidad]['Promedio por afiliado'].std()

        desvios_amedia = ((df[df["N° Prestador"] == prestador]["Promedio por afiliado"]) - (df[df["N° Prestador"] == prestador]["Media por especialidad"])) /desvio

        temporal[i[:7]]=desvios_amedia.iloc[0].round(2)

In [39]:
x = temporal.columns[1:]
y_prestador = temporal[:1].values[0][1:]
fig_farmacia = go.Figure()
fig_farmacia.add_trace(go.Bar(x = x ,
                     y = y_prestador,
                     name = "Prestador", base=0, text=y_prestador, textposition="auto", marker_color = "#45b6fe"
                         ))
fig_farmacia.update_traces(textposition="outside")


fig_farmacia.update_layout(title= "Desvíos en prescipciones de farmacia según media de especialidad",
                 showlegend= False)
fig_farmacia.update_yaxes(range=[-2,2],tickvals = [2, 1, 0, -1,-2], ticktext=["2 desvíos", "1 desvío", "Media especialidad","-1 desvío", "-2 desvíos"])